# 我们的国旗

上大学时我在学生会做过宣传委员，曾经办过一张板报，板报上画过一面国旗，结果上午抬出去，下午就被抬回来了，并被通报说画的国旗不标准，要求重新设计。当时并不明白什么原因，所以当时连夜重新设计了板报，把国旗改成了党旗。

后来才知道当时的国旗没有一心向党。
![](./image/qt_china_flag_3.png)

为了做好这个画国旗教程，我特意查了下国旗的尺寸(3:2)：
1. 2880x1920; 2. 2400x1600; 3. 1920x1280; 4. 1440x960; 5. 960x640; 6. 660x440

国旗，旗面红色，象征革命。左上方五颗红星，一星较大，居左，象征中国共产党； 4星较小，环拱大星之右，并各有一角正对大星中心，象征新中国成立时的工人阶级、农民阶级、城市小资产解决、民族资产阶级。5星象征革命人民大团结和人民对党的衷心拥护。

国旗上五星的位置有着很精确的定位，先将长方形国旗分成四份，并把左上角的小矩形再次拆分，长边分为15格，宽边分为10格，从左向右1-15，从上至下1-10。象征中国共产党的五角星位于(5,5)格，五角星所在圆直径等于6格长度；4颗小星分别位于(10,2),(12,4), (12,7), (10,9)格位置，五角星所在圆直径等于2格长度，4颗小星均有一角指向大五角星。

![](./image/qt_china_flag_1.png)

![](./image/qt_china_flag_2.png)


# 设计思路

为了让大家明白国旗的含义，我决定让大家看到国旗的绘制动画，并增加辅助线以让大家明白四小星向着大星的含义。所以：
1. 绘制一个660x440的矩形
2. 矩形中间画线，拆分为4份
3. 左上角矩形拆分为10行15列
4. 找到大五角星圆心，画圆
5. 找到小五角星圆心，画圆
6. 4个小圆圆形到大圆圆心画线
7. 相同方向画出5个五角星
8. 4个小五角星进行旋转使一角与圆心连线重合
9. 矩形填充红色
10. 五角星填充黄色

# 在哪里画图、用什么画、怎么画

* 用什么画？ Qt中用`QPainter`进行绘画
* 画在哪？可以画在继承了`QPaintDevice`的类上，包括：QGLFramebufferObject, QGLPixelBuffer, QImage, QOpenGLPaintDevice, QPagedPaintDevice, QPaintDeviceWindow, QPicture, QPixmap, QSvgGenerator, 和 QWidget。所以继承自`QWidget`的控件均可画图，有很多，当然可以直接在Qwidget上绘画。
* 怎么画？QPainter 类提供了很多绘画相关的函数可被调用，例如：drawLine、drawImage、drawEllipse、drawPath、drawPoint等等。
* 什么时候画？drawLine等绘图函数并不会直接把图片画在控件上，而是需要重构paintEvent函数，所以需要创建依赖于QWidget的子类，并在子类中重构paintEvent函数。
* 哪些条件会触发paintEvent事件？1）窗口第一次执行时系统会自动产生一个绘制事件。2）调整窗口控件大小时，系统会自动产生一个绘制事件。3）窗口控件被其他窗口遮挡，然后又再次显示时会产生绘制事件。4）调用控件的update函数或repaint函数时会产生一个绘制事件。

# 绘制国旗


## 画图前的准备

1. 使用控件QWidget，长宽比例3：2，我们选择660x440的比例尺寸

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor

绘图类QPainter和画笔颜色需要导入。然后需要重新定义控件以重构绘图事件处理函数。我们定义了`QWidgetDraw`类，类中重新定义paintEvent函数，为了让绘图过程更简单，把绘图部分封装为函数`draw`。代码如下

In [2]:
class QWidgetDraw(QWidget):
    def __init__(self, parent = None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        qpainter = QPainter()
        qpainter.begin(self)
        self.draw(qpainter)
        qpainter.end()
    
    def draw(self, qpainter):
        qpainter.drawLine(20,20,100,100)
        print("u can draw")

draw函数中添加了一条日志，这样我们就可以很好的验证什么时候画图的问题了。定义完`QWidgetDraw`类后就需要进行实例化、配置窗口参数并显示窗口了。为了看到exec_()中做的事情，我们在show()后添加日志进行验证。同时drawLine中设定直线的起始坐标是(20,20)到(100,100)，所以还可以看到绘图坐标系的坐标原点位置。

In [3]:
app = QApplication([])

widgetDraw = QWidgetDraw()
widgetDraw.resize(660+40,440+40)
widgetDraw.show()
print("widgetDraw show done")

app.exec_()

widgetDraw show done
u can draw


0

从日志可以看出，程序是执行exec_()时自动创建了paintevent，之后才调用了draw函数。并且子啊屏幕上切换不同窗口后再次且回画图界面会再次触发paintevent，并执行draw函数
```
# 执行日志
widgetDraw show done
u can draw
u can draw
u can draw
```

## 绘制矩形

绘制矩形只需要修改draw函数中的绘图命令即可。为了执行正确，我们重新定义QWidgetDraw类如下：

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor
from PyQt5.QtCore import QRectF

class QWidgetDraw(QWidget):
    def __init__(self, parent = None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        qpainter = QPainter()
        qpainter.begin(self)
        self.draw(qpainter)
        qpainter.end()

    def draw(self, qpainter):
        qpainter.drawRect(20, 20, 660, 440) # 绘制国旗外边框

app = QApplication([])

widgetDraw = QWidgetDraw()
widgetDraw.resize(660+40,440+40)
widgetDraw.show()

app.exec_()

0

然后我们需要把矩形窗口分隔为4份，我们可以定义一个等分函数在先把大的矩形划分为4等分，然后再把左上角的矩形划分为10x15份，代码如下：

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor
from PyQt5.QtCore import QRectF

class QWidgetDraw(QWidget):
    def __init__(self, parent = None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        qpainter = QPainter()
        qpainter.begin(self)
        self.draw(qpainter)
        qpainter.end()

    def draw(self, qpainter):
        qpainter.drawRect(20, 20, 660, 440) # 绘制国旗外边框
        
        # 矩形四等分，然后左上角矩形做10x15等分
        self.drawDivide(qpainter, 20, 20, 2, 2, 660, 440)
        self.drawDivide(qpainter, 20, 20, 15, 10, 330, 220)
    
    # 左上角矩形绘制10x15的分割
    def drawDivide(self, qpainter, x, y, wCnt, hCnt, w, h):
        step = w/wCnt
        for num in range(wCnt):
            qpainter.drawLine(x+num*step, y, x+num*step, y+h)
        
        step = h/hCnt
        for num in range (hCnt):
            qpainter.drawLine(x, y+num*step, x+w, y+num*step)

app = QApplication([])

widgetDraw = QWidgetDraw()
widgetDraw.resize(660+40,440+40)
widgetDraw.show()

app.exec_()

qwidgetdraw.h
```
#ifndef QWIDGETDRAW_H
#define QWIDGETDRAW_H

#include <QWidget>

class QWidgetDraw : public QWidget
{
    Q_OBJECT
public:
    explicit QWidgetDraw(QWidget *parent = nullptr);

    void draw(QPainter *painter);

    void drawDivide(QPainter *painter, int startX, int startY, int width, int hight, int wCnt, int hCnt);
    void drawStar(QPainter *painter, int x, int y, int radius, int rotate);

private:
    void paintEvent(QPaintEvent *event) override;
signals:

};

#endif // QWIDGETDRAW_H

```


qwidgetdraw.c
```
#include "qwidgetdraw.h"
#include <QPaintEvent>
#include <QPainter>
#include <QtMath>

QWidgetDraw::QWidgetDraw(QWidget *parent) : QWidget(parent)
{
    this->setWindowTitle("我们的");
    // 1.2880x1920; 2. 2400x1600; 3. 1920x1280; 4. 1440x960; 5. 960x640; 6. 660x440
    this->resize(660+100, 440+100);
    // 必须完成todo屏幕上显示团队信息，个人名字/学号
    // 必须完成todo界面上显示时间
    // 拓展：时间显示用LCDnumber替代QLabel
}

void QWidgetDraw::paintEvent(QPaintEvent *event)
{
    QPainter *painter = new QPainter();
    // 启动绘图
    painter->begin(this);
    this->draw(painter);
    // 绘图结束
    painter->end();
}

void QWidgetDraw::draw(QPainter *painter)
{
    int step;
    // 配置画笔参数
    painter->setPen(QPen(Qt::black, Qt::DashLine));
//    painter->setBrush(QBrush(Qt::red, Qt::SolidPattern));
    // 画国旗外框
    painter->drawRect(50, 50, 660, 440);
    // 水平垂直2等分
    this->drawDivide(painter, 50, 50, 660, 440, 2, 2);
    // 左上角10x15等分
    this->drawDivide(painter, 50, 50, 660/2, 440/2, 15, 10);
    // 绘制大圆
    step = 330/15;
    painter->drawEllipse(50 + step*2, 50 + step*2, step*6, step*6);
    painter->drawEllipse(50 + step*9, 50 + step*1, step*2, step*2);
    // todo
    // 必须完成其他4个小圆
    // 拓展：画大的五角星
}

void QWidgetDraw::drawDivide(QPainter *painter, int startX, int startY, int width, int hight, int wCnt, int hCnt)
{
    int step = width/wCnt;
    int cnt = 0;

    // 水平方向等分
    for(cnt = 0; cnt < wCnt; cnt++)
        painter->drawLine(startX+step*cnt, startY, startX+step*cnt, startY+hight);

    // 垂直方向等分
    step = hight/hCnt;
    for(cnt = 0; cnt < hCnt; cnt++)
        painter->drawLine(startX, startY+step*cnt, startX+width, startY+step*cnt);
}


```

main.c
```
#include <QApplication>
#include <QWidget>
#include <QPainter>
#include "qwidgetdraw.h"

int main(int argc, char * argv[])
{
    QApplication a(argc, argv);

    QWidgetDraw drawWin;
    drawWin.show();

    return a.exec();
}


```

## 绘制圆形

Qt中圆形的绘制是用QPainter的drawEllipse函数实现的，此函数本身绘制椭圆的，但是如果椭圆的宽和高一样的话就是圆了，我们可以在draw函数中将一个大圆和四个小圆一起画出。

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor
from PyQt5.QtCore import QRectF

class QWidgetDraw(QWidget):
    def __init__(self, parent = None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        qpainter = QPainter()
        qpainter.begin(self)
        self.draw(qpainter)
        qpainter.end()

    def draw(self, qpainter):
        qpainter.drawRect(20, 20, 660, 440) # 绘制国旗外边框
        
        # 矩形四等分，然后左上角矩形做10x15等分
        self.drawDivide(qpainter, 20, 20, 2, 2, 660, 440)
        self.drawDivide(qpainter, 20, 20, 15, 10, 330, 220)
        
        # 绘制五角星的同心圆
        step = 330/15
        qpainter.drawEllipse(20 + step*5, 20 + step*5, step*6, step*6) # 大圆，位于(5,5)格点，半径3个栅格
    
    # 左上角矩形绘制10x15的分割
    # x,y 为起始坐标
    # wCnt，hCnt分别是宽高分割份数
    # w, h是待分割区域的宽和高
    def drawDivide(self, qpainter, x, y, wCnt, hCnt, w, h):
        step = w/wCnt
        for num in range(wCnt):
            qpainter.drawLine(x+num*step, y, x+num*step, y+h)
        
        step = h/hCnt
        for num in range (hCnt):
            qpainter.drawLine(x, y+num*step, x+w, y+num*step)

app = QApplication([])

widgetDraw = QWidgetDraw()
widgetDraw.resize(660+40,440+40)
widgetDraw.show()

app.exec_()

0

绘图发现drawEllipse函数前两个参数并不是圆心所在位置，而是在与圆相切正方形的左上角位置。所以修改代码为:
```
qpainter.drawEllipse(20 + step*(5-3), 20 + step*(5-3), step*6, step*6) # 大圆，位于(5,5)格点，半径3个栅格
qpainter.drawEllipse(20 + step*(10-1), 20 + step*(2-1), step*2, step*2) # 小圆，位于(10,2)格点，半径1个栅格
qpainter.drawEllipse(20 + step*(12-1), 20 + step*(4-1), step*2, step*2) # 小圆，位于(12,4)格点，半径1个栅格
qpainter.drawEllipse(20 + step*(12-1), 20 + step*(7-1), step*2, step*2) # 小圆，位于(12,7)格点，半径1个栅格
qpainter.drawEllipse(20 + step*(10-1), 20 + step*(9-1), step*2, step*2) # 小圆，位于(10,9)格点，半径1个栅格
```
4颗小星分别位于(10,2),(12,4), (12,7), (10,9)格

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor
from PyQt5.QtCore import QRectF

class QWidgetDraw(QWidget):
    def __init__(self, parent = None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        qpainter = QPainter() # 使用新创建的类
        qpainter.begin(self)
        self.draw(qpainter)
        qpainter.end()

    def draw(self, qpainter):
        qpainter.drawRect(20, 20, 660, 440) # 绘制国旗外边框
        
        # 矩形四等分，然后左上角矩形做10x15等分
        self.drawDivide(qpainter, 20, 20, 2, 2, 660, 440)
        self.drawDivide(qpainter, 20, 20, 15, 10, 330, 220)
        
        # 绘制五角星的同心圆
        step = 330/15
        qpainter.drawEllipse(20 + step*(5-3), 20 + step*(5-3), step*6, step*6) # 大圆，位于(5,5)格点，半径3个栅格
        qpainter.drawEllipse(20 + step*(10-1), 20 + step*(2-1), step*2, step*2) # 小圆，位于(10,2)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(12-1), 20 + step*(4-1), step*2, step*2) # 小圆，位于(12,4)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(12-1), 20 + step*(7-1), step*2, step*2) # 小圆，位于(12,7)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(10-1), 20 + step*(9-1), step*2, step*2) # 小圆，位于(10,9)格点，半径1个栅格
    
    # 左上角矩形绘制10x15的分割
    # x,y 为起始坐标
    # wCnt，hCnt分别是宽高分割份数
    # w, h是待分割区域的宽和高
    def drawDivide(self, qpainter, x, y, wCnt, hCnt, w, h):
        step = w/wCnt
        for num in range(wCnt):
            qpainter.drawLine(x+num*step, y, x+num*step, y+h)
        
        step = h/hCnt
        for num in range (hCnt):
            qpainter.drawLine(x, y+num*step, x+w, y+num*step)

app = QApplication([])

widgetDraw = QWidgetDraw()
widgetDraw.resize(660+40,440+40)
widgetDraw.show()

app.exec_()

0

## 圆心画线

获取圆心坐标， 用drawLine函数即可完成绘制斜线

## 画五角星

五角星会有很多种画法，这里只讨论一种，那就是以圆的半径为高，画一个等边三角形，然后让这个三角形绕圆心旋转4次，实际上就是画5个三角形。这样三角形区域进行填充后画出来的就是五角星了。

我们设计一个drawStar函数，入口参数为圆的圆心和圆的半径：已知五角星的5个角的角度为180/5=36，根据三角函数可以计算出三角形三个顶点坐标，假设圆心所在位置为坐标原点。

方法：
* 根据几何原理计算五角星顶角为36度
* 根据五角星所在圆半径计算，三角形底边长度 L= R*tan(pi*36/2/360) * 2
* 假定圆心为坐标原点，换算三角形三个点所在坐标[(0, -R), (-L/2, 0), (L/2, 0)]
* 配置画笔颜色，画刷颜色
* 坐标系做平移、旋转
* 绘制三角形
* 重复旋转、绘制三角形，完成五角星绘制

In [ ]:
    def drawStar(self, painter, x, y, radius, rotate, fillStyle):
        # 三角形顶角的一半
        degree = 2*math.pi*(180/10)/360
        
        # 计算三角形顶点坐标
        points = [QPoint(0, -radius), QPoint(-radius*math.tan(degree), 0), QPoint(radius*math.tan(degree), 0)]
        
        painter.translate(x,y) # 坐标系平移
        painter.rotate(rotate) # 坐标系旋转
        self.drawTriangle(painter, points, fillStyle) # 绘图
        painter.resetTransform() # 坐标系复位
        
    def drawTriangle(self, painter, points, fillStyle):
        painter.drawPolygon(QPolygon(points), fillStyle)

`radius*math.tan(degree)`例如正切函数计算直角三角形的另外一边，实际上就是对应的2坐标位置。

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor, QPolygon
from PyQt5.QtCore import QRectF, QPoint, Qt
import math

class QWidgetDraw(QWidget):
    def __init__(self, parent = None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        qpainter = QPainter() # 使用新创建的类
        qpainter.begin(self)
        self.draw(qpainter)
        qpainter.end()

    def draw(self, qpainter):
        qpainter.drawRect(20, 20, 660, 440) # 绘制国旗外边框

        # 矩形四等分，然后左上角矩形做10x15等分
        self.drawDivide(qpainter, 20, 20, 2, 2, 660, 440)
        self.drawDivide(qpainter, 20, 20, 15, 10, 330, 220)
        
        # 绘制五角星的同心圆
        step = 330/15
        qpainter.drawEllipse(20 + step*(5-3), 20 + step*(5-3), step*6, step*6) # 大圆，位于(5,5)格点，半径3个栅格
        qpainter.drawEllipse(20 + step*(10-1), 20 + step*(2-1), step*2, step*2) # 小圆，位于(10,2)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(12-1), 20 + step*(4-1), step*2, step*2) # 小圆，位于(12,4)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(12-1), 20 + step*(7-1), step*2, step*2) # 小圆，位于(12,7)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(10-1), 20 + step*(9-1), step*2, step*2) # 小圆，位于(10,9)格点，半径1个栅格
        
        # 绘制大五角星
        self.drawStar(qpainter, 20 + step*(5), 20 + step*(5), step*3, 0, 0)
        

    
    # 左上角矩形绘制10x15的分割
    # x,y 为起始坐标
    # wCnt，hCnt分别是宽高分割份数
    # w, h是待分割区域的宽和高
    def drawDivide(self, qpainter, x, y, wCnt, hCnt, w, h):
        step = w/wCnt
        for num in range(wCnt):
            qpainter.drawLine(x+num*step, y, x+num*step, y+h)
        
        step = h/hCnt
        for num in range (hCnt):
            qpainter.drawLine(x, y+num*step, x+w, y+num*step)
            
    def drawStar(self, painter, x, y, radius, rotate, fillStyle):
        # 三角形顶角的一半
        degree = 2*math.pi*(180/10)/360
        
        # 计算三角形顶点坐标
        points = [QPoint(0, -radius), QPoint(-radius*math.tan(degree), 0), QPoint(radius*math.tan(degree), 0)]
        
        # 设置画刷颜色
        painter.setBrush(QColor("#FAF408"))
        painter.setPen(QColor("#FAF408"));
        
        # 调整坐标系，并绘制三角形
        painter.resetTransform() # 坐标系复位
        painter.translate(x,y) # 坐标系平移
        painter.rotate(rotate) # 坐标系旋转
        self.drawTriangle(painter, points, fillStyle) # 绘图
        painter.resetTransform() # 坐标系复位
        
    def drawTriangle(self, painter, points, fillStyle):
        painter.drawPolygon(QPolygon(points), fillStyle)

app = QApplication([])

widgetDraw = QWidgetDraw()
widgetDraw.resize(660+40,440+40)
widgetDraw.show()

app.exec_()

0

程序运行结果如下如：
![](./image/qt_painter_draw_triangle.png)

## 任务1. 绘制完整五角星

参考答案如下：

In [ ]:
    def drawStar(self, painter, x, y, radius, rotate, fillStyle):
        degree = 2*math.pi*(180/10)/360
        points = [QPoint(0, -radius), QPoint(-radius*math.tan(degree), 0), QPoint(radius*math.tan(degree), 0)] # (0,0)在三角形底边中点，三角形高为radius，三角形顶角为36度
        
        painter.setBrush(QColor("#FAF408"))
        painter.setPen(QColor("#FAF408"));
        painter.resetTransform() # 坐标系复位
        painter.translate(x,y) # 坐标系平移
        painter.rotate(rotate) # 坐标系旋转
        for cnt in range(5):
            self.drawTriangle(painter, points, fillStyle) # 绘图
            painter.rotate(72) # 坐标系旋转
        painter.resetTransform() # 坐标系复位

## 添加4颗小五角星完成国旗绘制

提示：
1. 根据三角函数计算小圆与大圆圆心连线与水平方向夹角，此夹角作为`drawStar`函数中入口参数`rotate`的值。
2. 国旗旗面红色可在绘制第一个矩形时配置为：painter.setBrush(QColor("#F40002"));

参考答案如下：

In [1]:
from PyQt5.QtWidgets import QApplication, QWidget
from PyQt5.QtGui import QPainter, QColor, QPolygon
from PyQt5.QtCore import QRectF, QPoint, Qt
import math

class QWidgetDraw(QWidget):
    def __init__(self, parent = None):
        super(QWidgetDraw, self).__init__(parent)
        
    def paintEvent(self, event):
        qpainter = QPainter() # 使用新创建的类
        qpainter.begin(self)
        self.draw(qpainter)
        qpainter.end()

    def draw(self, qpainter):
        # 重绘国旗旗面底色
        qpainter.resetTransform() # 坐标系复位
        qpainter.setBrush(QColor("#F40002"))
        qpainter.setPen(QColor("#F40002"))
        qpainter.drawRect(20, 20, 660, 440) # 绘制国旗外边框
        qpainter.resetTransform() # 坐标系复位
        
        # 矩形四等分，然后左上角矩形做10x15等分
        self.drawDivide(qpainter, 20, 20, 2, 2, 660, 440)
        self.drawDivide(qpainter, 20, 20, 15, 10, 330, 220)
        
        # 绘制五角星的同心圆
        step = 330/15
        qpainter.drawEllipse(20 + step*(5-3), 20 + step*(5-3), step*6, step*6) # 大圆，位于(5,5)格点，半径3个栅格
        qpainter.drawEllipse(20 + step*(10-1), 20 + step*(2-1), step*2, step*2) # 小圆，位于(10,2)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(12-1), 20 + step*(4-1), step*2, step*2) # 小圆，位于(12,4)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(12-1), 20 + step*(7-1), step*2, step*2) # 小圆，位于(12,7)格点，半径1个栅格
        qpainter.drawEllipse(20 + step*(10-1), 20 + step*(9-1), step*2, step*2) # 小圆，位于(10,9)格点，半径1个栅格
        
        # 绘制大五角星
        self.drawStar(qpainter, 20 + step*(5), 20 + step*(5), step*3, 0, 0)
        
        # 绘制小五角星
        self.drawStar(qpainter, 20 + step*(10), 20 + step*(2), step*1, -(90+math.atan(3/5)/math.pi*180), 0)
        self.drawStar(qpainter, 20 + step*(12), 20 + step*(4), step*1, -(90+math.atan(1/7)/math.pi*180), 0)
        self.drawStar(qpainter, 20 + step*(12), 20 + step*(7), step*1, -(90-math.atan(2/7)/math.pi*180), 0)
        self.drawStar(qpainter, 20 + step*(10), 20 + step*(9), step*1, -(90-math.atan(4/5)/math.pi*180), 0)
        
    # 左上角矩形绘制10x15的分割
    # x,y 为起始坐标
    # wCnt，hCnt分别是宽高分割份数
    # w, h是待分割区域的宽和高
    def drawDivide(self, qpainter, x, y, wCnt, hCnt, w, h):
        step = w/wCnt
        for num in range(wCnt):
            qpainter.drawLine(x+num*step, y, x+num*step, y+h)
        
        step = h/hCnt
        for num in range (hCnt):
            qpainter.drawLine(x, y+num*step, x+w, y+num*step)
            
    def drawStar(self, painter, x, y, radius, rotate, fillStyle):
        # 三角形顶角的一半
        degree = 2*math.pi*(180/10)/360
        
        # 计算三角形顶点坐标
        points = [QPoint(0, -radius), QPoint(-radius*math.tan(degree), 0), QPoint(radius*math.tan(degree), 0)]
        
        # 设置画刷颜色
        painter.setBrush(QColor("#FAF408"))
        painter.setPen(QColor("#FAF408"))
        
        # 调整坐标系，并绘制三角形
        painter.resetTransform() # 坐标系复位
        painter.translate(x,y) # 坐标系平移
        painter.rotate(rotate) # 坐标系旋转
        for cnt in range(5):
            self.drawTriangle(painter, points, fillStyle) # 绘图
            painter.rotate(72) # 坐标系旋转
        painter.resetTransform() # 坐标系复位
        
    def drawTriangle(self, painter, points, fillStyle):
        painter.drawPolygon(QPolygon(points), fillStyle)

app = QApplication([])

widgetDraw = QWidgetDraw()
widgetDraw.resize(660+40,440+40)
widgetDraw.show()

app.exec_()

0